In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("~/PycharmProjects/Diabetes/finalNew.csv", sep=',')
a1c_data = data[['Patient_ID', 'PerformedDate', 'a1c_Res']]
a1c_data = a1c_data.dropna()

a1c_data = a1c_data.sort_values(by='PerformedDate', ascending=True)

grouped_patients = a1c_data.groupby("Patient_ID")
grouped_lists = grouped_patients["a1c_Res"].apply(list)
grouped_lists = grouped_lists.reset_index()
grouped_lists['a1c_Res'] = grouped_lists['a1c_Res'].apply(lambda x: list(set(x)))
data = grouped_lists[['a1c_Res']]
print(data)

                                                  a1c_Res
0                          [81.0, 82.0, 84.0, 86.0, 87.0]
1       [66.0, 99.0, 101.0, 6.0, 102.0, 105.0, 106.0, ...
2                                                  [48.0]
3       [64.0, 65.0, 66.0, 68.0, 69.0, 6.0, 7.0, 73.0,...
4       [99.0, 68.0, 69.0, 7.0, 104.0, 73.0, 10.0, 105...
...                                                   ...
127492                                        [71.0, 7.0]
127493                                 [96.0, 91.0, 87.0]
127494                                 [72.0, 74.0, 78.0]
127495                                       [67.0, 68.0]
127496                                        [75.0, 7.0]

[127497 rows x 1 columns]


In [ ]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

a = [0,0,0]
b = [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
for index, row in data.iterrows() :
    #raw_seq = data['Albumin/Creatinine_Res'].at[index]
    if len(data['a1c_Res'].at[index]) > 15 :
        seq = data['a1c_Res'].at[index]
        # choose a number of time steps
        n_steps = 3
        # split into samples
        X, y = split_sequence(seq, n_steps)
        y = y.reshape((-1, 1))
        a = np.vstack((a, X))
        b = np.vstack((b, y))
    else:
        continue
X = np.delete(a, (0), axis=0)
y = b[10:, :]
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
model.fit(X_train, y_train, epochs=200, verbose=0)
#for index, row in data.iterrows() :
# demonstrate prediction

In [ ]:
for index, row in data.iterrows() :
    if len(data['BP_res'].at[index]) > 15 :
        seq = data['BP_res'].at[index]
        #x_input = np.asarray(X_test[-4:-1])
        #x_input = X_test.reshape((1, n_steps, n_features))
        yhat = model.predict(X_test, verbose=0)
        y_actual = np.asarray(y_test[-1:])
        model.evaluate(X_train, y_train)